# PM 5 Task H POD Truncation


In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse.linalg import eigs
import time

# Import sonar simulation functions
from getParam_Sonar import getParam_Sonar
from eval_f_Sonar import eval_f_Sonar
from eval_g_Sonar import eval_g_Sonar
from eval_u_Sonar import eval_u_Sonar_20, eval_u_Sonar_20_const
from simpleLeapFrog import LeapfrogSolver

## Full Order Model

In [ ]:
Lx, Lz = 8e3, 2e3      
Nx, Nz = 601, 151  
f0 = 20               
eval_u = eval_u_Sonar_20_const      

N = Nx * Nz
n = 2 * N

print(f"Domain: {Lx}m x {Lz}m")
print(f"Grid: {Nx} x {Nz} = {N:,} spatial points")
print(f"State dimension: 2N = {n:,}")

# Get parameters (this builds A and B)
p, x_start, t_start, t_stop, max_dt_FE = getParam_Sonar(
    Nx, Nz, Lx, Lz, UseSparseMatrices=True, BC=True
)

t_sim = t_stop 

# Extract system matrices
A = p['A']
B = p['B']

print(f"\nGrid spacing: dx = {p['dx']:.4f}m, dz = {p['dz']:.4f}m")
print(f"Sound speed: c = {p['c']} m/s")
print(f"Max stable dt: {max_dt_FE:.2e}s")
print(f"Simulation time: {t_stop:.2f} s")

# Wavelength check
wavelength = p['c'] / f0
print(f"\nAt f₀ = {f0} Hz:")
print(f"  Wavelength: λ = {wavelength:.2f}m")
print(f"  Points per wavelength: {wavelength/p['dx']:.0f}")
print(f"  Domain coverage: {Lx/wavelength:.1f}λ x {Lz/wavelength:.1f}λ")

### Input function

In [ ]:
# Scale by cell area (same as PM1)
def eval_u_scaled(t):
    return (p['dx'] * p['dz']) * eval_u(t)

# Visualize pulse
t_test = np.linspace(0, t_sim, 1000)   
u_test = [eval_u(t) for t in t_test]

plt.figure(figsize=(10, 3))
plt.plot(t_test * 1000, u_test, 'b-', linewidth=1.5)
plt.xlabel('Time (ms)')
plt.ylabel('u(t)')
plt.title(f'Input Pulse: {f0} Hz Gaussian-windowed sine')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Output matrix

In [ ]:
# Use hydrophones from getParam_Sonar
n_phones = p['hydrophones']['n_phones']
h_z = p['hydrophones']['z_pos']
h_x_indices = p['hydrophones']['x_indices']

# C matrix selects pressure at hydrophone locations
C = np.zeros((n_phones, n))
for i, hx in enumerate(h_x_indices):
    idx = hx * Nz + h_z
    C[i, N + idx] = 1.0  

print(f"Hydrophones: {n_phones} receivers")
for i, hx in enumerate(h_x_indices):
    print(f"  H{i+1}: x = {hx * p['dx']:.0f}m")

Run full order sim for snapshots

In [ ]:
'''
# takes 6 min 42 s
# Full-order simulation
dt = max_dt_FE * 0.1
num_steps = int(np.ceil(t_sim / dt))

print(f"Running full-order simulation...")
print(f"  Duration: {t_sim*1000:.0f} ms")
print(f"  Timestep: {dt*1e6:.1f} μs")
print(f"  Steps: {num_steps:,}")

t0 = time.perf_counter()
X_full, t_full = LeapfrogSolver(
    eval_f_Sonar, x_start, p, eval_u_scaled, num_steps, dt, visualize=False
)
full_time = time.perf_counter() - t0

print(f"\n✓ Full-order complete in {full_time:.1f}s")
print(f"  Snapshot matrix: {X_full.shape}")

# Verify BC
surface_p = X_full[N::Nz, :]
print(f"  Surface pressure max: {np.abs(surface_p).max():.2e} (should be ~0)")

# Extract hydrophone outputs
y_full = (C @ X_full).T  # (n_times, n_phones)
'''

In [ ]:
'''
# takes 9 min 31 s
# Save full-order simulation
import numpy as np

print("Saving full-order simulation...")

np.savez_compressed('pod_outputs/full_order_601x151.npz',
    X_full=X_full,
    t_full=t_full,
    y_full=y_full,
    full_time=full_time,
    Nx=Nx, Nz=Nz, Lx=Lx, Lz=Lz,
    dt=dt, t_sim=t_sim
)

print(f"✓ Saved to full_order_601x151.npz")
'''

In [ ]:
# takes 1 min 
# Load full-order simulation
data = np.load('pod_outputs/full_order_601x151.npz')

X_full = data['X_full']
t_full = data['t_full']
y_full = data['y_full']
full_time = float(data['full_time'])
dt = float(data['dt'])

print(f"Loaded X_full: {X_full.shape}")
print(f"Original solve time: {full_time:.1f}s")

Compute POD basis

In [ ]:
'''
# can run if grid is small enough
# SVD of snapshot matrix
print("Computing POD basis (SVD)...")
t0 = time.perf_counter()
U, S, Vh = np.linalg.svd(X_full, full_matrices=False)
svd_time = time.perf_counter() - t0
print(f"✓ SVD complete in {svd_time:.2f}s")
'''

In [ ]:
'''
# this takes 4 min 22 s
from sklearn.decomposition import TruncatedSVD

# Subsample timesteps
skip = 10
X_sub = X_full[:, ::skip]
print(f"Original: {X_full.shape}")
print(f"Subsampled: {X_sub.shape} (every {skip}th snapshot)")

k = 200

print(f"\nComputing SVD (k={k})...")
t0 = time.perf_counter()
U_full, S_full, Vh = np.linalg.svd(X_sub, full_matrices=False)
svd_time = time.perf_counter() - t0
print(f"✓ Done in {svd_time:.1f}s")

# Truncate to k modes
U = U_full[:, :k]
S = S_full[:k]

print(f"\nSingular values: {S[0]:.2e} to {S[-1]:.2e}")
print(f"Ratio S[0]/S[-1]: {S[0]/S[-1]:.0f}x")

# Energy captured
energy = np.cumsum(S**2) / np.sum(S_full**2)
print(f"\nEnergy in top {k} modes: {energy[-1]*100:.2f}%")
'''

In [ ]:
'''
# this takes 6 s
# Save all
np.savez_compressed('pod_outputs/svd_601x151.npz',
    U=U, S=S, k=k,
    S_full=S_full,
    svd_time=svd_time, 
    skip=skip,
    Nx=Nx, Nz=Nz, Lx=Lx, Lz=Lz
)

print(f"✓ Saved (U: {U.shape}, S: {S.shape})")
print(f"✓ Saved to svd_601x151.npz")
'''

In [ ]:
# Load SVD
data = np.load('pod_outputs/svd_601x151.npz')
U = data['U']
S = data['S']
k = int(data['k'])
Nx, Nz = int(data['Nx']), int(data['Nz'])
Lx, Lz = float(data['Lx']), float(data['Lz'])

print(f"Loaded U: {U.shape}, S: {S.shape}")
print(f"Grid: {Nx} x {Nz}")

# Energy analysis
if 'S_full' in data:
    S_full = data['S_full']
    energy = np.cumsum(S**2) / np.sum(S_full**2)
else:
    energy = np.cumsum(S**2) / np.sum(S**2)
    
print(f"Top {k} modes energy: {energy[-1]*100:.2f}%")

POD energy analysis

In [ ]:
cumulative_energy = np.cumsum(S**2) / np.sum(S**2)
n_90 = np.searchsorted(cumulative_energy, 0.90) + 1
n_99 = np.searchsorted(cumulative_energy, 0.99) + 1
n_999 = np.searchsorted(cumulative_energy, 0.999) + 1

print(f"Energy thresholds:")
print(f"  90% energy:   {n_90} modes")
print(f"  99% energy:   {n_99} modes")
print(f"  99.9% energy: {n_999} modes")

# Plot singular value decay
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.semilogy(S, 'b.-', markersize=3)
ax1.set_xlabel('Mode index')
ax1.set_ylabel('Singular value')
ax1.set_title('Singular value decay')
ax1.grid(True, alpha=0.3)

ax2.plot(cumulative_energy * 100, 'b.-', markersize=3)
ax2.axhline(90, color='r', linestyle='--', label='90%')
ax2.axhline(99, color='g', linestyle='--', label='99%')
ax2.axhline(99.9, color='orange', linestyle='--', label='99.9%')
ax2.set_xlabel('Number of modes')
ax2.set_ylabel('Cumulative energy (%)')
ax2.legend()
ax2.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

Stability check on q values

In [ ]:
q_values = list(range(1, 201, 1))  

results = []

print(f"{'q':<6} {'Max Re(λ)':<15} {'Status'}")
print("-" * 40)

for q in q_values:
    if q > U.shape[1]:
        break
    
    Phi_q = U[:, :q]
    A_pod_q = Phi_q.T @ (A @ Phi_q)
    
    eigs = np.linalg.eigvals(A_pod_q)
    max_re = np.real(eigs).max()
    
    # Stable if max Re(λ) ≤ 0
    stable = max_re <= 0
    status = "✓ Stable" if stable else "✗ Unstable"
    
    results.append({
        'q': q,
        'max_re': max_re,
        'stable': stable
    })
    
    print(f"{q:<6} {max_re:<15.4e} {status}")


qs = [r['q'] for r in results]
max_res = [r['max_re'] for r in results]
stable = [r['stable'] for r in results]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Max Re(λ) vs q
colors = ['green' if s else 'red' for s in stable]
axes[0].scatter(qs, max_res, c=colors, s=50)
axes[0].axhline(0, color='k', linestyle='--', lw=1, label='Stability boundary (Re(λ)=0)')
axes[0].set_xlabel('Number of modes (q)')
axes[0].set_ylabel('Max Re(λ)')
axes[0].set_title('ROM Eigenvalues vs q (green=stable, red=unstable)')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Binary stability map
axes[1].bar(qs, [1 if s else 0 for s in stable], 
            color=['green' if s else 'red' for s in stable], width=4, alpha=0.7)
axes[1].set_xlabel('Number of modes (q)')
axes[1].set_ylabel('Stable (1) / Unstable (0)')
axes[1].set_title('Stability Map (Re(λ) ≤ 0)')
axes[1].set_ylim([0, 1.2])
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary
stable_qs = [r['q'] for r in results if r['stable']]
unstable_qs = [r['q'] for r in results if not r['stable']]

print("\n" + "="*60)
print("STABILITY SUMMARY (Re(λ) ≤ 0)")
print("="*60)
print(f"Stable q values: {stable_qs}")
print(f"Unstable q values: {unstable_qs}")
if stable_qs:
    print(f"\nRecommended: q={max(stable_qs)} (highest stable)")
print("="*60)

Build POD ROM - sparse projection (don't convert A to dense)

In [ ]:
#q_pod = n_999
q_pod = max(stable_qs)
print(f"Building POD-ROM with q={q_pod} modes ({cumulative_energy[q_pod-1]*100:.2f}% energy)")

Phi = U[:, :q_pod]

# Sparse projection - A stays sparse!
print("Projecting A (sparse)...")
A_Phi = A @ Phi           # (n × q) sparse @ dense = dense
A_pod = Phi.T @ A_Phi     # (q × n) @ (n × q) = (q × q)

B_pod = Phi.T @ B.toarray()
C_pod = C @ Phi
x0_pod = Phi.T @ x_start  # <-- Add this line

N = Nx * Nz
n = 2 * N

print(f"\nA_pod: {A_pod.shape}")
print(f"B_pod: {B_pod.shape}")
print(f"C_pod: {C_pod.shape}")
print(f"Compression: {100*q_pod/n:.4f}%")

# Check stability
eigs_pod = np.linalg.eigvals(A_pod)
max_re = np.real(eigs_pod).max()
print(f"\nMax Re(λ): {max_re:.2e} (should be ≤ 0)")
if max_re > 0:
    print(f"⚠ Unstable - growth factor over {t_sim:.1f}s: {np.exp(max_re * t_sim):.2f}x")
else:
    print("✓ Stable")

Eigenvalue analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Full complex plane
axes[0].scatter(eigs_pod.real, eigs_pod.imag, c='b', s=30, alpha=0.7)
axes[0].axvline(0, color='r', linestyle='--', lw=1, label='Stability boundary')
axes[0].set_xlabel('Re(λ)')
axes[0].set_ylabel('Im(λ)')
axes[0].set_title('POD-ROM Eigenvalues')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Zoom on real axis near zero
axes[1].scatter(eigs_pod.real, eigs_pod.imag, c='b', s=30, alpha=0.7)
axes[1].axvline(0, color='r', linestyle='--', lw=1)
x_margin = max(abs(eigs_pod.real).max() * 0.1, 0.1)
axes[1].set_xlim(-x_margin, x_margin)
axes[1].set_xlabel('Re(λ)')
axes[1].set_ylabel('Im(λ)')
axes[1].set_title('Zoom: Near stability boundary')
axes[1].grid(True, alpha=0.3)

# Histogram of real parts
axes[2].hist(eigs_pod.real, bins=30, edgecolor='black', alpha=0.7)
axes[2].axvline(0, color='r', linestyle='--', lw=2, label='Stability boundary')
axes[2].set_xlabel('Re(λ)')
axes[2].set_ylabel('Count')
axes[2].set_title('Distribution of Re(λ)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
print("Eigenvalue statistics:")
print(f"  Total modes: {len(eigs_pod)}")
print(f"  Max Re(λ): {eigs_pod.real.max():.4e}")
print(f"  Min Re(λ): {eigs_pod.real.min():.4e}")
print(f"  Unstable modes (Re > 0): {np.sum(eigs_pod.real > 0)}")

unstable = eigs_pod[eigs_pod.real > 0]
if len(unstable) > 0:
    print(f"\nUnstable eigenvalues:")
    for e in unstable[:10]:  # Show first 10 max
        print(f"    λ = {e.real:.4e} + {e.imag:.4e}j")
    if len(unstable) > 10:
        print(f"    ... and {len(unstable) - 10} more")
    
    # Growth rate analysis
    max_growth = eigs_pod.real.max()
    doubling_time = np.log(2) / max_growth if max_growth > 0 else np.inf
    print(f"\nInstability analysis:")
    print(f"  Max growth rate: {max_growth:.4e} /s")
    print(f"  Doubling time: {doubling_time:.2f} s")
    print(f"  Simulation time: {t_sim:.2f} s")
    print(f"  Growth factor over sim: {np.exp(max_growth * t_sim):.2f}x")
else:
    print("\n✓ All eigenvalues stable")

Simulate POD ROM and compare

In [ ]:
'''
def simulate_pod_leapfrog(A_pod, B_pod, C_pod, u_func, t_span, dt, x0=None):
    """Leapfrog integrator for POD-ROM"""
    t = np.arange(t_span[0], t_span[1] + dt, dt)
    n_steps = len(t)
    q = A_pod.shape[0]
    n_out = C_pod.shape[0]
    
    x = np.zeros((n_steps, q))
    y = np.zeros((n_steps, n_out))
    
    x_prev = x0.flatten().copy() if x0 is not None else np.zeros(q)
    x_curr = x_prev.copy()
    
    # RK4 bootstrap
    u0 = u_func(t[0])
    k1 = A_pod @ x_curr + B_pod.flatten() * u0
    k2 = A_pod @ (x_curr + 0.5*dt*k1) + B_pod.flatten() * u_func(t[0] + 0.5*dt)
    k3 = A_pod @ (x_curr + 0.5*dt*k2) + B_pod.flatten() * u_func(t[0] + 0.5*dt)
    k4 = A_pod @ (x_curr + dt*k3) + B_pod.flatten() * u_func(t[0] + dt)
    x_next = x_curr + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)
    
    x[0], y[0] = x_curr, C_pod @ x_curr
    x_prev, x_curr = x_curr.copy(), x_next.copy()
    
    for i in range(1, n_steps):
        x[i], y[i] = x_curr, C_pod @ x_curr
        if i < n_steps - 1:
            f_i = A_pod @ x_curr + B_pod.flatten() * u_func(t[i])
            x_next = x_prev + 2 * dt * f_i
            x_prev, x_curr = x_curr.copy(), x_next.copy()
    
    return t, y, x

# Run POD-ROM
x0_pod = Phi.T @ x_start

t0 = time.perf_counter()
t_pod, y_pod, x_pod = simulate_pod_leapfrog(A_pod, B_pod, C_pod, eval_u_scaled, (0, t_sim), dt, x0_pod)
pod_time = time.perf_counter() - t0

print(f"✓ POD-ROM complete in {pod_time:.3f}s")
print(f"  Speedup: {full_time/pod_time:.0f}x")
'''

In [ ]:
def simulate_pod_trapezoidal(A_pod, B_pod, C_pod, u_func, t_span, dt, x0=None):
    """Trapezoidal (Crank-Nicolson) integrator - A-stable for any Re(λ) ≤ 0"""
    t = np.arange(t_span[0], t_span[1] + dt, dt)
    n_steps = len(t)
    q = A_pod.shape[0]
    n_out = C_pod.shape[0]
    
    x = np.zeros((n_steps, q))
    y = np.zeros((n_steps, n_out))
    
    x_curr = x0.flatten().copy() if x0 is not None else np.zeros(q)
    B_flat = B_pod.flatten()
    
    # Precompute matrices for trapezoidal rule:
    # x_{n+1} = x_n + (dt/2) * (f_n + f_{n+1})
    # x_{n+1} = x_n + (dt/2) * (A x_n + B u_n + A x_{n+1} + B u_{n+1})
    # (I - dt/2 * A) x_{n+1} = (I + dt/2 * A) x_n + (dt/2) * B * (u_n + u_{n+1})
    
    I = np.eye(q)
    LHS = I - (dt/2) * A_pod  # Left-hand side matrix
    RHS_mat = I + (dt/2) * A_pod  # Right-hand side matrix
    
    # Precompute LU factorization for efficiency
    from scipy.linalg import lu_factor, lu_solve
    LU = lu_factor(LHS)
    
    x[0] = x_curr
    y[0] = C_pod @ x_curr
    
    for i in range(1, n_steps):
        u_prev = u_func(t[i-1])
        u_curr = u_func(t[i])
        
        rhs = RHS_mat @ x_curr + (dt/2) * B_flat * (u_prev + u_curr)
        x_curr = lu_solve(LU, rhs)
        
        x[i] = x_curr
        y[i] = C_pod @ x_curr
    
    return t, y, x


print(f"Testing q={q_pod} with Trapezoidal...")
t0 = time.perf_counter()
t_pod, y_pod, x_pod = simulate_pod_trapezoidal(A_pod, B_pod, C_pod, eval_u_scaled, (0, t_sim), dt, x0_pod)
trap_time = time.perf_counter() - t0

print(f"\nTrapezoidal time: {trap_time:.3f}s")
print(f"Full-order time:  {full_time:.1f}s")
print(f"Speedup:          {full_time/trap_time:.0f}x")
print(f"\nMax |y_pod|:  {np.abs(y_pod).max():.2e}")
print(f"Max |y_full|: {np.abs(y_full).max():.2e}")

# Compute errors
errors = []
for i in range(n_phones):
    y_pod_interp = np.interp(t_full, t_pod, y_pod[:, i])
    rel_err = np.linalg.norm(y_full[:, i] - y_pod_interp) / (np.linalg.norm(y_full[:, i]) + 1e-12)
    errors.append(rel_err)

print(f"\nHydrophone errors:")
for i, err in enumerate(errors):
    print(f"  H{i+1}: {100*err:.2f}%")
print(f"Average error: {100*np.mean(errors):.2f}%")

In [ ]:
# Hydrophone comparison
errors = []

fig, axes = plt.subplots(n_phones, 2, figsize=(14, 3*n_phones))
if n_phones == 1:
    axes = axes.reshape(1, -1)

for i in range(n_phones):
    y_pod_interp = np.interp(t_full, t_pod, y_pod[:, i])
    err = y_full[:, i] - y_pod_interp
    rel_err = np.linalg.norm(err) / (np.linalg.norm(y_full[:, i]) + 1e-12)
    errors.append(rel_err)
    
    axes[i, 0].plot(t_full*1000, y_full[:, i], 'b-', lw=1.5, label='Full', alpha=0.7)
    axes[i, 0].plot(t_pod*1000, y_pod[:, i], 'r--', lw=1.5, label=f'POD (q={q_pod})')
    axes[i, 0].set_ylabel('Pressure (Pa)')
    axes[i, 0].set_title(f'H{i+1}: Full vs POD')
    axes[i, 0].legend()
    axes[i, 0].grid(True, alpha=0.3)
    
    axes[i, 1].plot(t_full*1000, err, 'g-', lw=1)
    axes[i, 1].set_ylabel('Error (Pa)')
    axes[i, 1].set_title(f'H{i+1} Error (rel: {100*rel_err:.2f}%)')
    axes[i, 1].grid(True, alpha=0.3)

axes[-1, 0].set_xlabel('Time (ms)')
axes[-1, 1].set_xlabel('Time (ms)')
plt.tight_layout()
plt.show()

''''
# if leapfrog
# Summary
print("=" * 60)
print("POD-ROM RESULTS")
print("=" * 60)
print(f"Full-order: {n:,} DOFs, {full_time:.1f}s")
print(f"POD-ROM:    {q_pod} DOFs, {pod_time:.3f}s")
print(f"Speedup:    {full_time/pod_time:.0f}x")
print(f"\nRelative errors:")
for i, err in enumerate(errors):
    print(f"  H{i+1}: {100*err:.4f}%")
print("=" * 60)
'''

# Summary
print("=" * 60)
print("POD-ROM RESULTS")
print("=" * 60)
print(f"Full-order: {n:,} DOFs, {full_time:.1f}s")
print(f"POD-ROM:    {q_pod} DOFs, {trap_time:.3f}s")
print(f"Speedup:    {full_time/trap_time:.0f}x")
print(f"\nRelative errors:")
for i, err in enumerate(errors):
    print(f"  H{i+1}: {100*err:.4f}%")
print("=" * 60)

Energy comparison for stability

In [ ]:
# this takes 6 min to run
'''
print("Comparing energy evolution: Full-order vs POD-ROM...")

# Full-order energy
def compute_energy(X, Nx, Nz):
    N = Nx * Nz
    v = X[:N, :]
    p = X[N:, :]
    return np.sum(v**2, axis=0) + np.sum(p**2, axis=0)

energy_full = compute_energy(X_full, Nx, Nz)

# POD-ROM energy (reconstruct state from modal coordinates)
X_pod_reconstructed = Phi @ x_pod.T  # (n, n_steps)
energy_pod = compute_energy(X_pod_reconstructed, Nx, Nz)

# Print stats
print(f"\nFull-order energy:")
print(f"  Peak:   {energy_full.max():.2e}")
print(f"  Final:  {energy_full[-1]:.2e}")
print(f"  Change: {(energy_full[-1]/energy_full.max() - 1)*100:+.2f}%")

print(f"\nPOD-ROM energy:")
print(f"  Peak:   {energy_pod.max():.2e}")
print(f"  Final:  {energy_pod[-1]:.2e}")
print(f"  Change: {(energy_pod[-1]/energy_pod.max() - 1)*100:+.2f}%")

# Check for blowup
if energy_pod[-1] > energy_full[-1] * 10:
    print("\n⚠ POD-ROM shows energy blowup!")
else:
    print("\n✓ POD-ROM energy matches full-order")

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Log scale
axes[0].semilogy(t_full * 1000, energy_full, 'b-', lw=1.5, label='Full-order')
axes[0].semilogy(t_pod * 1000, energy_pod, 'r--', lw=1.5, label='POD-ROM')
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Total Energy')
axes[0].set_title('Energy Evolution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Normalized (look for divergence)
axes[1].plot(t_full * 1000, energy_full / energy_full.max(), 'b-', lw=1.5, label='Full-order')
axes[1].plot(t_pod * 1000, energy_pod / energy_pod.max(), 'r--', lw=1.5, label='POD-ROM')
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Normalized Energy')
axes[1].set_title('Normalized Energy (look for growth)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary
print("\n" + "="*50)
print("STABILITY SUMMARY")
print("="*50)
print(f"Damping coefficient α = {p['alpha']:.2e}")
print(f"Full-order energy change: {(energy_full[-1]/energy_full.max() - 1)*100:+.2f}%")
print(f"POD-ROM energy change:    {(energy_pod[-1]/energy_pod.max() - 1)*100:+.2f}%")
print("="*50)

# Plot comparison
fig, ax = plt.subplots(figsize=(10, 5))

ax.semilogy(t_full * 1000, energy_full, 'b-', lw=1.5, label='Full-order')
ax.semilogy(t_pod * 1000, energy_pod, 'r--', lw=1.5, label='POD-ROM')
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Total Energy')
ax.set_title('Energy Evolution: Full-order vs POD-ROM')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
'''

That's expected for POD with traveling waves. The errors appear on ray paths because:

POD modes are global and stationary - they span the whole domain
Waves are localized and moving - sharp features traveling through space
Truncation smooths sharp wavefronts - error concentrates where gradients are highest

The ray paths are where the acoustic energy is - direct wave, surface reflection, seafloor reflection. POD struggles most with these sharp, localized features.

In [ ]:
# Reconstruct full state from POD
X_pod_reconstructed = Phi @ x_pod.T

# Check surface pressure (should be ~0 for p=0 BC)
N = Nx * Nz
surface_p_full = X_full[N::Nz, :]      # Every Nz-th pressure node (j=0)
surface_p_pod = X_pod_reconstructed[N::Nz, :]

print("Surface pressure (p=0 at z=0):")
print(f"  Full-order max: {np.abs(surface_p_full).max():.2e}")
print(f"  POD-ROM max:    {np.abs(surface_p_pod).max():.2e}")

# Vertical array visualization
def extract_vertical_array(p, X, x_pos=None):
    Nx, Nz = p['Nx'], p['Nz']
    N = Nx * Nz
    if x_pos is None:
        x_pos = Nx // 2
    z_indices = list(range(0, Nz))  # Include surface!
    pressure = np.zeros((len(z_indices), X.shape[1]))
    for di, zi in enumerate(z_indices):
        flat_idx = x_pos * Nz + zi
        pressure[di, :] = X[N + flat_idx, :]
    depths = np.array(z_indices) * p['dz']
    return pressure, depths

# Extract from both
pressure_full, depths = extract_vertical_array(p, X_full)
pressure_pod, _ = extract_vertical_array(p, X_pod_reconstructed)

# Match lengths
n_times = min(pressure_full.shape[1], pressure_pod.shape[1])
pressure_full = pressure_full[:, :n_times]
pressure_pod = pressure_pod[:, :n_times]
t_plot = t_full[:n_times]

t_ms = np.array(t_plot) * 1000
vmax = max(np.abs(pressure_full).max(), np.abs(pressure_pod).max())

fig, axes = plt.subplots(1, 3, figsize=(16, 6))

# Full-order
im0 = axes[0].imshow(pressure_full, aspect='auto', cmap='RdBu_r',
                      extent=[t_ms[0], t_ms[-1], depths[-1], depths[0]],
                      vmin=-vmax, vmax=vmax)
axes[0].axhline(p['sonar_iz'] * p['dz'], color='yellow', linestyle='--', lw=2, label='Source')
axes[0].axhline(0, color='cyan', linestyle='-', lw=2, label='Surface (p=0)')
axes[0].axhline(p['Lz'], color='brown', linestyle='-', lw=2, label='Seafloor')
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Depth (m)')
axes[0].set_title(f'Full-order ({full_time:.1f}s)')
axes[0].legend(loc='upper right', fontsize=8)
plt.colorbar(im0, ax=axes[0], label='Pressure (Pa)')

# POD-ROM
im1 = axes[1].imshow(pressure_pod, aspect='auto', cmap='RdBu_r',
                      extent=[t_ms[0], t_ms[-1], depths[-1], depths[0]],
                      vmin=-vmax, vmax=vmax)
axes[1].axhline(p['sonar_iz'] * p['dz'], color='yellow', linestyle='--', lw=2)
axes[1].axhline(0, color='cyan', linestyle='-', lw=2)
axes[1].axhline(p['Lz'], color='brown', linestyle='-', lw=2)
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Depth (m)')
#axes[1].set_title(f'POD-ROM q={q_pod} ({pod_time:.2f}s)')
axes[1].set_title(f'POD-ROM q={q_pod} ({trap_time:.2f}s)')
plt.colorbar(im1, ax=axes[1], label='Pressure (Pa)')

# Error
error = pressure_pod - pressure_full
err_max = np.abs(error).max()
im2 = axes[2].imshow(error, aspect='auto', cmap='RdBu_r',
                      extent=[t_ms[0], t_ms[-1], depths[-1], depths[0]],
                      vmin=-err_max, vmax=err_max)
axes[2].set_xlabel('Time (ms)')
axes[2].set_ylabel('Depth (m)')
rel_err = np.linalg.norm(error) / (np.linalg.norm(pressure_full) + 1e-12) * 100
axes[2].set_title(f'Error (rel: {rel_err:.2f}%)')
plt.colorbar(im2, ax=axes[2], label='Error (Pa)')

plt.suptitle('Boundary Condition Verification: Full vs POD-ROM', fontsize=12)
plt.tight_layout()
plt.show()

# Surface time series comparison
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(t_ms, pressure_full[0, :], 'b-', lw=1.5, label='Full-order (surface)')
ax.plot(t_ms, pressure_pod[0, :], 'r--', lw=1.5, label='POD-ROM (surface)')
ax.axhline(0, color='k', linestyle=':', alpha=0.5)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Pressure at surface (Pa)')
ax.set_title('Surface BC check: p(z=0) should be ≈ 0')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nSurface BC violation (RMS):")
print(f"  Full-order: {np.sqrt(np.mean(pressure_full[0, :]**2)):.2e} Pa")
print(f"  POD-ROM:    {np.sqrt(np.mean(pressure_pod[0, :]**2)):.2e} Pa")

Visualize

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def plot_comparison(Index):
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    N = Nx * Nz
    
    # Full-order pressure (single frame)
    p_full = X_full[N:, Index].reshape(Nx, Nz).T
    
    # POD pressure (reconstruct single frame on-the-fly)
    pod_idx = int(Index * len(t_pod) / len(t_full))
    pod_idx = min(pod_idx, len(t_pod) - 1)
    x_pod_frame = x_pod[pod_idx, :]  # Modal coordinates at this time
    X_pod_frame = Phi @ x_pod_frame   # Reconstruct single frame
    p_pod = X_pod_frame[N:].reshape(Nx, Nz).T
    
    # Error
    p_error = p_full - p_pod
    
    # Common colorscale
    vmax = max(np.abs(p_full).max(), np.abs(p_pod).max())
    vmin = -vmax
    
    # Full-order
    im0 = axes[0].imshow(p_full, aspect='auto', cmap='RdBu_r', vmin=vmin, vmax=vmax,
                          extent=[0, Lx, Lz, 0])
    axes[0].set_title(f'Full-order (t={t_full[Index]*1000:.1f} ms)')
    axes[0].set_xlabel('Range (m)')
    axes[0].set_ylabel('Depth (m)')
    plt.colorbar(im0, ax=axes[0], label='Pressure (Pa)')
    
    # POD-ROM
    im1 = axes[1].imshow(p_pod, aspect='auto', cmap='RdBu_r', vmin=vmin, vmax=vmax,
                          extent=[0, Lx, Lz, 0])
    axes[1].set_title(f'POD-ROM q={q_pod} (t={t_pod[pod_idx]*1000:.1f} ms)')
    axes[1].set_xlabel('Range (m)')
    axes[1].set_ylabel('Depth (m)')
    plt.colorbar(im1, ax=axes[1], label='Pressure (Pa)')
    
    # Error
    err_max = np.abs(p_error).max()
    im2 = axes[2].imshow(p_error, aspect='auto', cmap='RdBu_r', vmin=-err_max, vmax=err_max,
                          extent=[0, Lx, Lz, 0])
    rel_err = np.linalg.norm(p_error) / (np.linalg.norm(p_full) + 1e-12) * 100
    axes[2].set_title(f'Error (rel: {rel_err:.2f}%)')
    axes[2].set_xlabel('Range (m)')
    axes[2].set_ylabel('Depth (m)')
    plt.colorbar(im2, ax=axes[2], label='Error (Pa)')
    
    plt.tight_layout()
    plt.show()

# Create slider
slider = widgets.IntSlider(
    value=0, 
    min=0, 
    max=X_full.shape[1] - 1, 
    step=100,
    description='Time index'
)

ui = widgets.HBox([slider])
out = widgets.interactive_output(plot_comparison, {'Index': slider})
display(ui, out)

In [ ]:
# this takes 

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import Image

# Select frames (subsample for speed)
n_frames = 400 # 1000
frame_indices = np.linspace(0, (X_full.shape[1] - 1)/2, n_frames, dtype=int)

N = Nx * Nz

# Create figure - vertical stack, wide and short
fig, axes = plt.subplots(3, 1, figsize=(16, 6))

# Initialize with first frame
p_full = X_full[N:, 0].reshape(Nx, Nz).T
p_pod = (Phi @ x_pod[0, :])[N:].reshape(Nx, Nz).T
p_error = p_full - p_pod

# Get global colorscale from a few sample frames
vmax_global = 0
for idx in frame_indices[::10]:
    p_sample = X_full[N:, idx]
    vmax_global = max(vmax_global, np.abs(p_sample).max())

# Reduce colorscale to make colors darker/more saturated
vmax_plot = vmax_global * 0.1  # More saturated

im0 = axes[0].imshow(p_full, aspect='auto', cmap='RdBu_r', vmin=-vmax_plot, vmax=vmax_plot,
                      extent=[0, Lx, Lz, 0])
axes[0].set_xlabel('Range (m)')
axes[0].set_ylabel('Depth (m)')
plt.colorbar(im0, ax=axes[0], label='Pa')
title0 = axes[0].set_title('Full-order')

im1 = axes[1].imshow(p_pod, aspect='auto', cmap='RdBu_r', vmin=-vmax_plot, vmax=vmax_plot,
                      extent=[0, Lx, Lz, 0])
axes[1].set_xlabel('Range (m)')
axes[1].set_ylabel('Depth (m)')
plt.colorbar(im1, ax=axes[1], label='Pa')
title1 = axes[1].set_title(f'POD-ROM (q={q_pod})')

err_max = vmax_plot * .1
im2 = axes[2].imshow(p_error, aspect='auto', cmap='RdBu_r', vmin=-err_max, vmax=err_max,
                      extent=[0, Lx, Lz, 0])
axes[2].set_xlabel('Range (m)')
axes[2].set_ylabel('Depth (m)')
plt.colorbar(im2, ax=axes[2], label='Pa')
title2 = axes[2].set_title('Error')

plt.tight_layout()

def update(frame):
    idx = frame_indices[frame]
    pod_idx = int(idx * len(t_pod) / len(t_full))
    pod_idx = min(pod_idx, len(t_pod) - 1)
    
    # Get data
    p_full = X_full[N:, idx].reshape(Nx, Nz).T
    p_pod = (Phi @ x_pod[pod_idx, :])[N:].reshape(Nx, Nz).T
    p_error = p_full - p_pod
    
    # Update images
    im0.set_data(p_full)
    im1.set_data(p_pod)
    im2.set_data(p_error)
    
    # Update titles
    t_ms = t_full[idx] * 1000
    rel_err = np.linalg.norm(p_error) / (np.linalg.norm(p_full) + 1e-12) * 100
    title0.set_text(f'Full-order (t={t_ms:.0f} ms)')
    title1.set_text(f'POD-ROM q={q_pod} (t={t_ms:.0f} ms)')
    title2.set_text(f'Error ({rel_err:.1f}%)')
    
    return [im0, im1, im2, title0, title1, title2]

print("Creating animation...")
anim = FuncAnimation(fig, update, frames=n_frames, interval=50, blit=True)

# Save as GIF
print("Saving GIF (this may take a minute)...")
anim.save('pod_outputs/pod_comparison.gif', writer=PillowWriter(fps=20))
print("✓ Saved to pod_outputs/pod_comparison.gif")

plt.close()

# Display
Image(filename='pod_outputs/pod_comparison.gif')

In [ ]:
# Plot error vs time
errors_vs_time = []
for i, idx in enumerate(frame_indices):
    pod_idx = int(idx * len(t_pod) / len(t_full))
    pod_idx = min(pod_idx, len(t_pod) - 1)
    
    p_full = X_full[N:, idx]
    p_pod = (Phi @ x_pod[pod_idx, :])[N:]
    
    rel_err = np.linalg.norm(p_full - p_pod) / (np.linalg.norm(p_full) + 1e-12)
    errors_vs_time.append(rel_err * 100)

plt.figure(figsize=(10, 4))
plt.plot(t_full[frame_indices] * 1000, errors_vs_time, 'b-', lw=1.5)
plt.xlabel('Time (ms)')
plt.ylabel('Relative Error (%)')
plt.title('POD-ROM Error vs Time')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Max error: {max(errors_vs_time):.1f}% at t={t_full[frame_indices[np.argmax(errors_vs_time)]]*1000:.0f} ms")
print(f"Final error: {errors_vs_time[-1]:.1f}%")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

half_idx = len(t_full) // 2

for i in range(n_phones):
    y_pod_interp = np.interp(t_full[:half_idx], t_pod, y_pod[:, i])
    
    # Normalize by max signal amplitude (not pointwise)
    signal_max = np.abs(y_full[:half_idx, i]).max()
    abs_error = np.abs(y_full[:half_idx, i] - y_pod_interp)
    
    # Error as % of max signal
    rel_error = abs_error / signal_max * 100
    
    ax.plot(t_full[:half_idx] * 1000, rel_error, lw=1.5, label=f'H{i+1}', alpha=0.8)

ax.set_xlabel('Time (ms)')
ax.set_ylabel('Error (% of max signal)')
ax.set_title(f'Normalized Error - POD-ROM q={q_pod}')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Stability preserve

In [ ]:
'''
# =============================================================================
# STRUCTURE-PRESERVING POD-ROM (COTANGENT LIFT)
# =============================================================================

print("="*60)
print("STRUCTURE-PRESERVING POD-ROM (COTANGENT LIFT)")
print("="*60)

# Your system has structure:
#   dv/dt = -αv + Lp    (momentum)
#   dp/dt = v           (continuity)
#
# Key insight: Use SAME basis for v and p (cotangent lift)
# This ensures M_pv = I and preserves structure

# =============================================================================
# STEP 1: Separate velocity and pressure snapshots
# =============================================================================

N = Nx * Nz
V_snapshots = X_full[:N, :]      # velocity (first half)
P_snapshots = X_full[N:, :]      # pressure (second half)

print(f"Velocity snapshots: {V_snapshots.shape}")
print(f"Pressure snapshots: {P_snapshots.shape}")

# =============================================================================
# STEP 2: POD on pressure (use same basis for both)
# =============================================================================

# POD for pressure
U_p, S_p, _ = np.linalg.svd(P_snapshots, full_matrices=False)
energy_p = np.cumsum(S_p**2) / np.sum(S_p**2)

q_sp = np.searchsorted(energy_p, 0.999) + 1
print(f"\nUsing q = {q_sp} modes (99.9% pressure energy)")

# COTANGENT LIFT: Same basis for both v and p
Phi_shared = U_p[:, :q_sp]
Phi_v = Phi_shared
Phi_p = Phi_shared

# =============================================================================
# STEP 3: Build structure-preserving ROM
# =============================================================================

# Extract L from A matrix structure: A = [[-αI, L], [I, 0]]
alpha = p['alpha']
A_dense = A.toarray()
L_mat = A_dense[:N, N:]  # Upper-right block

print(f"\nL_mat shape: {L_mat.shape}")
print(f"L_mat symmetry: {np.linalg.norm(L_mat - L_mat.T) / np.linalg.norm(L_mat):.2e}")

B_full = B.toarray()
B_v = B_full[:N, :]   # Input to velocity equation
B_p = B_full[N:, :]   # Input to pressure equation

# Build reduced matrices
M_vv = Phi_v.T @ Phi_v           # = I (orthonormal)
M_pv = Phi_p.T @ Phi_v           # = I (same basis!)
L_vp = Phi_v.T @ L_mat @ Phi_p   # Coupling

B_v_r = Phi_v.T @ B_v
B_p_r = Phi_p.T @ B_p

print(f"\nReduced matrices:")
print(f"  ||M_vv - I|| = {np.linalg.norm(M_vv - np.eye(q_sp)):.2e} (should be ~0)")
print(f"  ||M_pv - I|| = {np.linalg.norm(M_pv - np.eye(q_sp)):.2e} (should be ~0)")

# =============================================================================
# STEP 4: Assemble structure-preserving A_sp
# =============================================================================

# Since M_vv = M_pv = I:
#   dv_r/dt = -α v_r + L_vp p_r + B_v_r u
#   dp_r/dt = v_r

A_sp = np.block([
    [-alpha * np.eye(q_sp),  L_vp],
    [np.eye(q_sp),           np.zeros((q_sp, q_sp))]
])

B_sp = np.vstack([B_v_r, B_p_r])

# C matrix: extract pressure at hydrophones
C_p = C[:, N:]  # pressure part of C
C_sp = C_p @ Phi_p

print(f"\nStructure-preserving ROM:")
print(f"  A_sp: {A_sp.shape}")
print(f"  B_sp: {B_sp.shape}")
print(f"  C_sp: {C_sp.shape}")
print(f"  Total ROM dimension: {2*q_sp}")

# =============================================================================
# STEP 5: Check stability
# =============================================================================

eigs_sp = np.linalg.eigvals(A_sp)
max_re_sp = np.real(eigs_sp).max()

print(f"\nStability:")
print(f"  Standard POD max Re(λ): {np.real(eigs_pod).max():.4e}")
print(f"  Structure-preserving max Re(λ): {max_re_sp:.4e}")

if max_re_sp <= 1e-10:
    print("  ✓ Structure-preserving ROM is stable!")
elif max_re_sp < np.real(eigs_pod).max():
    print("  ⚠ More stable than standard POD")
else:
    print("  ⚠ Similar stability to standard POD")
'''

In [ ]:
'''
# =============================================================================
# STEP 6: Simulate structure-preserving ROM
# =============================================================================

def simulate_sp_leapfrog(A_sp, B_sp, C_sp, u_func, t_span, dt, x0=None):
    """Leapfrog for structure-preserving ROM"""
    t = np.arange(t_span[0], t_span[1] + dt, dt)
    n_steps = len(t)
    q2 = A_sp.shape[0]  # 2*q_sp
    n_out = C_sp.shape[0]
    
    x = np.zeros((n_steps, q2))
    y = np.zeros((n_steps, n_out))
    
    x_prev = x0.flatten().copy() if x0 is not None else np.zeros(q2)
    x_curr = x_prev.copy()
    
    # RK4 bootstrap
    u0 = u_func(t[0])
    k1 = A_sp @ x_curr + B_sp.flatten() * u0
    k2 = A_sp @ (x_curr + 0.5*dt*k1) + B_sp.flatten() * u_func(t[0] + 0.5*dt)
    k3 = A_sp @ (x_curr + 0.5*dt*k2) + B_sp.flatten() * u_func(t[0] + 0.5*dt)
    k4 = A_sp @ (x_curr + dt*k3) + B_sp.flatten() * u_func(t[0] + dt)
    x_next = x_curr + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)
    
    x[0], y[0] = x_curr, C_sp @ x_curr[q2//2:]  # Output from pressure part
    x_prev, x_curr = x_curr.copy(), x_next.copy()
    
    for i in range(1, n_steps):
        x[i] = x_curr
        y[i] = C_sp @ x_curr[q2//2:]  # Output from pressure part
        if i < n_steps - 1:
            f_i = A_sp @ x_curr + B_sp.flatten() * u_func(t[i])
            x_next = x_prev + 2 * dt * f_i
            x_prev, x_curr = x_curr.copy(), x_next.copy()
    
    return t, y, x

# Initial conditions
v0_r = Phi_v.T @ x_start[:N]
p0_r = Phi_p.T @ x_start[N:]
x0_sp = np.concatenate([v0_r, p0_r])

t0 = time.perf_counter()
t_sp, y_sp, x_sp = simulate_sp_leapfrog(A_sp, B_sp, C_sp, eval_u_scaled, (0, t_sim), dt, x0_sp)
sp_time = time.perf_counter() - t0

print(f"\n✓ Structure-preserving ROM complete in {sp_time:.3f}s")
'''


In [ ]:
'''
# =============================================================================
# STEP 7: Compare all three methods
# =============================================================================

fig, axes = plt.subplots(n_phones, 1, figsize=(14, 2.5*n_phones), sharex=True)
if n_phones == 1:
    axes = [axes]

errors_pod = []
errors_sp = []

for i in range(n_phones):
    y_pod_interp = np.interp(t_full, t_pod, y_pod[:, i])
    y_sp_interp = np.interp(t_full, t_sp, y_sp[:, i])
    
    err_pod = np.linalg.norm(y_full[:, i] - y_pod_interp) / (np.linalg.norm(y_full[:, i]) + 1e-12)
    err_sp = np.linalg.norm(y_full[:, i] - y_sp_interp) / (np.linalg.norm(y_full[:, i]) + 1e-12)
    errors_pod.append(err_pod)
    errors_sp.append(err_sp)
    
    axes[i].plot(t_full*1000, y_full[:, i], 'b-', lw=1.5, label='Full', alpha=0.7)
    axes[i].plot(t_pod*1000, y_pod[:, i], 'r--', lw=1.5, label=f'POD (q={q_pod})')
    axes[i].plot(t_sp*1000, y_sp[:, i], 'g:', lw=2, label=f'SP-POD (q={2*q_sp})')
    axes[i].set_ylabel('Pressure (Pa)')
    axes[i].set_title(f'H{i+1}: POD err={100*err_pod:.1f}%, SP err={100*err_sp:.1f}%')
    axes[i].legend(loc='upper right')
    axes[i].grid(True, alpha=0.3)

axes[-1].set_xlabel('Time (ms)')
plt.suptitle('Full vs Standard POD vs Structure-Preserving POD')
plt.tight_layout()
plt.show()

# Eigenvalue comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].scatter(eigs_pod.real, eigs_pod.imag, c='r', s=30, alpha=0.7, label='Standard POD')
axes[0].scatter(eigs_sp.real, eigs_sp.imag, c='g', s=30, alpha=0.7, label='SP-POD')
axes[0].axvline(0, color='k', linestyle='--', lw=1)
axes[0].set_xlabel('Re(λ)')
axes[0].set_ylabel('Im(λ)')
axes[0].set_title('Eigenvalue Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Zoom
margin = 0.1
axes[1].scatter(eigs_pod.real, eigs_pod.imag, c='r', s=30, alpha=0.7, label='Standard POD')
axes[1].scatter(eigs_sp.real, eigs_sp.imag, c='g', s=30, alpha=0.7, label='SP-POD')
axes[1].axvline(0, color='k', linestyle='--', lw=1)
axes[1].set_xlim(-margin, margin)
axes[1].set_xlabel('Re(λ)')
axes[1].set_ylabel('Im(λ)')
axes[1].set_title('Zoom: Near stability boundary')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary
print("="*60)
print("COMPARISON: Standard POD vs Structure-Preserving POD")
print("="*60)
print(f"{'Method':<20} {'DOFs':<8} {'Time (s)':<10} {'Max Re(λ)':<12} {'Avg Error':<10}")
print("-"*60)
print(f"{'Standard POD':<20} {q_pod:<8} {pod_time:<10.3f} {np.real(eigs_pod).max():<12.2e} {100*np.mean(errors_pod):.2f}%")
print(f"{'SP-POD':<20} {2*q_sp:<8} {sp_time:<10.3f} {max_re_sp:<12.2e} {100*np.mean(errors_sp):.2f}%")
print("="*60)
'''

In [ ]:
'''
# The culprit: M_pv ≠ I means v and p bases are misaligned
print(f"||M_pv - I|| = {np.linalg.norm(M_pv - np.eye(q_sp)):.2f}")
'''